In [ ]:
import requests

In [ ]:
r = requests.get('https://www.google.com')
r.content

In [ ]:
url = 'https://samples.openweathermap.org/data/2.5/weather?q=London,uk&appid=b6907d289e10d714a6e88b30761fae22'
r = requests.get(url)

In [ ]:
r.text

In [ ]:
weather = r.json()
weather

In [ ]:
weather['weather']

In [ ]:
weather['weather'][0]

In [ ]:
weather['weather'][0]['description']

In [ ]:
api_key = '19ce16fc754aa52681ce41f2de654b29'
api_url_prefix = 'https://api.gateway.attomdata.com/propertyapi/v1.0.0/'

In [ ]:
headers = {
    'Accept': 'application/json',
    'apikey': api_key
}

In [ ]:
params = {
    'address1': '115 Mulberry Street', 
    'address2': 'New York, NY'
}
url = "{}saleshistory/detail".format(api_url_prefix)
url

In [ ]:
r = requests.get(url, params=params, headers=headers)

In [ ]:
r.text

In [ ]:
property_sales_data = r.json()

In [ ]:
property_sales_data

In [ ]:
sale_history = property_sales_data['property'][0]['salehistory']
sale_history

In [ ]:
for sale in sale_history:
    if 'saleTransDate' in sale and 'amount' in sale and 'saleamt' in sale['amount']:
        formatted_text = "{} sold for {}".format(sale['saleTransDate'], sale['amount']['saleamt'])
        print(formatted_text)

In [ ]:
params = {
    'postalcode': '90210', 
    'page': 1,
    'pagesize': 100
}
url = "{}property/address".format(api_url_prefix)
url

In [ ]:
r = requests.get(url, params=params, headers=headers)

properties_in_zip = r.json()

In [ ]:
properties_in_zip

In [ ]:
properties = properties_in_zip['property']
for prop in properties:
    print(prop['address']['oneLine'])

In [ ]:
import pandas as pd


properties = properties_in_zip['property']
data = []
for prop in properties:
    row = {}
    row['Address'] = prop['address']['line1']
    row['City'] = prop['address']['locality']
    row['Zip'] = prop['address']['postal1']
    row['Lat'] = prop['location']['latitude']
    row['Long'] = prop['location']['longitude']
    data.append(row)
    
data

In [ ]:
df = pd.DataFrame.from_dict(data)
df

In [ ]:
url = "{}property/address".format(api_url_prefix)

def get_all_properties_in_zip(zipcode, page_size = 100):
    data = []
    properties_downloaded = 0 
    total_properties = None
    
    params = {
        'postalcode': zipcode,
        'page': 1,
        'pagesize': page_size
    }
        
    page = 1
    while total_properties is None or properties_downloaded < total_properties:
        r = requests.get(url, params=params, headers=headers)
        properties_in_zip = r.json()
        properties = properties_in_zip['property']
        total_properties = properties_in_zip['status']['total']

        for prop in properties:
            row = {}
            row['Address'] = prop['address']['line1']
            row['City'] = prop['address']['locality']
            row['Zip'] = prop['address']['postal1']
            row['Lat'] = prop['location']['latitude']
            row['Long'] = prop['location']['longitude']
            data.append(row)
            
        properties_downloaded = properties_downloaded + page_size
        params['page'] = params['page'] + 1
        print('Got {} properties out of {} total'.format(properties_downloaded, total_properties))
        
    df = pd.DataFrame.from_dict(data)
    return df
    

In [ ]:
df = get_all_properties_in_zip(90210)
df